# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
import tensorflow as tf

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM, CuDNNLSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

from numpy.random import seed
from tensorflow import set_random_seed


C:\Users\controlslab\Anaconda3\envs\gpu_tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0.2
#l1_lambda_regularization = 0.2

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, dropout=0.2, name='lstm1'))
    model.add(CuDNNLSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm1'))
    model.add(Dropout(0.2))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
#min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
scaler  = StandardScaler()

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    model = RULmodel_LSTM(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model

In [6]:
num_features = len(selected_features)

shape = (window_size, num_features)
model = get_compiled_model(shape, model_type='lstm')
tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', model, lib_type='keras', data_handler=dHandler_cmaps)

# Load Data

In [7]:
tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 125

In [8]:
# #Load sequenced data (do not unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = scaler
tModel.data_scaler = None

tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0)
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 30, 14)
(17731, 1)
Testing data (X, y)
(100, 30, 14)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[[-1.72172539 -0.13425518 -0.92593596 ... -0.78170979  1.34849274
    1.19442705]
  [-1.06177971  0.21152849 -0.64372587 ... -0.78170979  1.01652793
    1.23692196]
  [-0.66181262 -0.41316559 -0.52595315 ... -2.07309423  0.73989059
    0.50342281]
  ...
  [-0.66181262 -1.25315894 -1.08926228 ... -2.07309423  1.0718554
   -0.05178246]
  [-1.54174021 -0.92857898 -0.88260449 ... -0.13601757  0.62923565
    0.64661219]
  [-0.96178794  0.48880785 -1.42813661 ... -2.07309423  1.29316527
    1.12052285]]

 [[-1.06177971  0.21152849 -0.64372587 ... -0.78170979  1.01652793
    1.23692196]
  [-0.66181262 -0.41316559 -0.52595315 ... -2.07309423  0.73989059
    0.50342281]
  [-0.66181262 -1.2

# Train the model and test some Tunable Model functionalities

In [9]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = (tModel.data_handler.sequence_length, nFeatures)
model = get_compiled_model(shape, model_type='lstm')
tModel.change_model('ModelRUL_RNN_1', model, 'keras')

tModel.epochs = 100

tModel.train_model(learningRate_scheduler=lrate, verbose=1)

Epoch 1/100
17731/17731 [==============================] - 3s 159us/step - loss: 8124.7990 - mean_squared_error: 8120.3017
Epoch 2/100
17731/17731 [==============================] - 0s 12us/step - loss: 7349.1129 - mean_squared_error: 7344.1392
Epoch 3/100
17731/17731 [==============================] - 0s 12us/step - loss: 6273.7442 - mean_squared_error: 6267.9084
Epoch 4/100
17731/17731 [==============================] - 0s 12us/step - loss: 5267.1569 - mean_squared_error: 5260.1987
Epoch 5/100
17731/17731 [==============================] - 0s 12us/step - loss: 4307.9284 - mean_squared_error: 4299.6841
Epoch 6/100
17731/17731 [==============================] - 0s 12us/step - loss: 3421.1964 - mean_squared_error: 3411.5036
Epoch 7/100
17731/17731 [==============================] - 0s 12us/step - loss: 2634.1178 - mean_squared_error: 2622.8905
Epoch 8/100
17731/17731 [==============================] - 0s 11us/step - loss: 1996.1576 - mean_squared_error: 1983.3752
Epoch 9/100
17731/17731

17731/17731 [==============================] - 0s 12us/step - loss: 238.0044 - mean_squared_error: 215.9240
Epoch 69/100
17731/17731 [==============================] - 0s 12us/step - loss: 235.0965 - mean_squared_error: 213.0250
Epoch 70/100
17731/17731 [==============================] - 0s 12us/step - loss: 236.5366 - mean_squared_error: 214.4736
Epoch 71/100
17731/17731 [==============================] - 0s 12us/step - loss: 236.2461 - mean_squared_error: 214.1879
Epoch 72/100
17731/17731 [==============================] - 0s 12us/step - loss: 234.7740 - mean_squared_error: 212.7216
Epoch 73/100
17731/17731 [==============================] - 0s 12us/step - loss: 236.6503 - mean_squared_error: 214.6067
Epoch 74/100
17731/17731 [==============================] - 0s 12us/step - loss: 239.8113 - mean_squared_error: 217.7718
Epoch 75/100
17731/17731 [==============================] - 0s 12us/step - loss: 238.4205 - mean_squared_error: 216.3893
Epoch 76/100
17731/17731 [===================

In [10]:
tModel.evaluate_model(['rhs', 'rmse'], round=2)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)

#print(tModel.y_predicted)
#print(tModel.y_predicted_rounded)
#print(tModel.y_test)

for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 937us/step
scores
Engine 1, Predicted RUL 122.89850616455078, Rounded RUL 122.0, Real RUL 112.0
Engine 2, Predicted RUL 117.25055694580078, Rounded RUL 117.0, Real RUL 98.0
Engine 3, Predicted RUL 43.832054138183594, Rounded RUL 43.0, Real RUL 69.0
Engine 4, Predicted RUL 83.31526947021484, Rounded RUL 83.0, Real RUL 82.0
Engine 5, Predicted RUL 97.1010971069336, Rounded RUL 97.0, Real RUL 91.0
Engine 6, Predicted RUL 117.3116683959961, Rounded RUL 117.0, Real RUL 93.0
Engine 7, Predicted RUL 113.59843444824219, Rounded RUL 113.0, Real RUL 91.0
Engine 8, Predicted RUL 85.09982299804688, Rounded RUL 85.0, Real RUL 95.0
Engine 9, Predicted RUL 119.50027465820312, Rounded RUL 119.0, Real RUL 111.0
Engine 10, Predicted RUL 85.80584716796875, Rounded RUL 85.0, Real RUL 96.0
Engine 11, Predicted RUL 71.91180419921875, Rounded RUL 71.0, Real RUL 97.0
Engine 12, Predicted RUL 112.26528930664062, Rounded RUL 112.0, Real RUL 124.0
Engine 13, Predicte